In [ ]:
! pip3 install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.0 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset


dataset = load_dataset("tapaco", "en")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/158053 [00:00<?, ? examples/s]

In [ ]:
print(dataset["train"][0])

{'paraphrase_set_id': '1', 'sentence_id': '416554', 'paraphrase': 'I ate the cheese.', 'lists': ['907', '4000', '6677', '7361', '7415'], 'tags': [''], 'language': 'en'}


In [ ]:
dataset["train"] = dataset["train"].select_columns(["paraphrase_set_id", "paraphrase"])

In [ ]:
print(dataset["train"][0])

{'paraphrase_set_id': '1', 'paraphrase': 'I ate the cheese.'}


In [ ]:
import pandas as pd

df = pd.DataFrame(dataset["train"])

In [ ]:
df

,paraphrase_set_id,paraphrase
0,1,I ate the cheese.
1,1,I eat cheese.
2,1,I'm eating a yogurt.
3,1,I'm eating cheese.
4,1,I'm having some cheese.
...,...,...
158048,843414,All of you are welcome to attend my party alon...
158049,844191,I wouldn't like being a judge.
158050,844191,I wouldn't want to be a judge.
158051,844308,To attend is important!


In [ ]:
import nltk
from nltk import pos_tag, word_tokenize
nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')
import re

for index, row in df.iterrows():
  row["paraphrase"] = row["paraphrase"].lower()
  row["paraphrase"] = word_tokenize(row["paraphrase"])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
df

,paraphrase_set_id,paraphrase
0,1,"[i, ate, the, cheese, .]"
1,1,"[i, eat, cheese, .]"
2,1,"[i, 'm, eating, a, yogurt, .]"
3,1,"[i, 'm, eating, cheese, .]"
4,1,"[i, 'm, having, some, cheese, .]"
...,...,...
158048,843414,"[all, of, you, are, welcome, to, attend, my, p..."
158049,844191,"[i, would, n't, like, being, a, judge, .]"
158050,844191,"[i, would, n't, want, to, be, a, judge, .]"
158051,844308,"[to, attend, is, important, !]"


In [ ]:
paraphrase_words = []
sentence = []
for index, row in df.iterrows():
  #creates dataframes with the same paraphrases set id
  df_same_paraphrases = df[df['paraphrase_set_id'] == row['paraphrase_set_id']]
  #iterate over paraphrase sets
  for index2, row2 in df_same_paraphrases.iterrows():
    if len(row['paraphrase']) == len(row2['paraphrase']) and row['paraphrase'] != row2['paraphrase'] :
      same_words = list(set(row['paraphrase']) & set(row2['paraphrase']))
      if len(same_words) == (len(row['paraphrase'])-1):
        # identify, mark and extract paraphrase pairs
        synonyms = list(set(row['paraphrase']) ^ set(row2['paraphrase']))
        if len(synonyms) == 2:
          targetwordList = list(set(row['paraphrase']) & set(synonyms))
          targetword = targetwordList[0]
          marked_targetword = "*" + targetword + "*"
          item = row['paraphrase']
          item = list(map(lambda x: x.replace(targetword, marked_targetword), item))
          sentence.append(item)
          print(row['paraphrase'])
          print(row2['paraphrase'])
          print(synonyms)
          print(item)
          paraphrase = list(set(row2['paraphrase']) & set(synonyms))
          print(paraphrase)
          paraphrase_words.append(paraphrase[0])
          print("_____________")
        else:
          pass
      else:
        pass
    else:
      pass

Streaming output truncated to the last 5000 lines.
['somebody']
_____________
['i', "'d", 'rather', 'help', 'somebody', 'else', '.']
['i', "'d", 'rather', 'help', 'someone', 'else', '.']
['someone', 'somebody']
['i', "'d", 'rather', 'help', '*somebody*', 'else', '.']
['someone']
_____________
['tom', 'must', "'ve", 'gone', 'fishing', '.']
['tom', 'must', 'have', 'gone', 'fishing', '.']
['have', "'ve"]
['tom', 'must', "*'ve*", 'gone', 'fishing', '.']
['have']
_____________
['tom', 'must', 'have', 'gone', 'fishing', '.']
['tom', 'must', "'ve", 'gone', 'fishing', '.']
["'ve", 'have']
['tom', 'must', '*have*', 'gone', 'fishing', '.']
["'ve"]
_____________
['i', 'actually', 'play', 'piano', 'quite', 'well', '.']
['i', 'actually', 'play', 'piano', 'pretty', 'well', '.']
['quite', 'pretty']
['i', 'actually', 'play', 'piano', '*quite*', 'well', '.']
['pretty']
_____________
['i', 'actually', 'play', 'piano', 'pretty', 'well', '.']
['i', 'actually', 'play', 'piano', 'quite', 'well', '.']
['quit

In [ ]:
print(len(sentence))
print(len(paraphrase_words))

62600
62600


In [ ]:
df_paraphrases = pd.DataFrame(list(zip(sentence, paraphrase_words)),
               columns =['sentence', 'paraphrase_word'])

In [ ]:
df_paraphrases

,sentence,paraphrase_word
0,"[i, ate, *the*, cheese, .]",some
1,"[i, *eat*, some, cheese, .]",ate
2,"[i, ate, *some*, cheese, .]",the
3,"[i, *ate*, some, cheese, .]",eat
4,"[it, *'s*, monday, .]",is
...,...,...
62595,"[she, watched, him, *drawing*, a, picture, .]",draw
62596,"[what, 's, your, favorite, *horror*, movie, ?]",scary
62597,"[what, 's, your, favorite, *scary*, movie, ?]",horror
62598,"[*globalise*, or, die, .]",globalize


In [ ]:
for index, row in df_paraphrases.iterrows():
  row["sentence"] = " ".join(row["sentence"])

In [ ]:
df_paraphrases

,sentence,paraphrase_word
0,i ate *the* cheese .,some
1,i *eat* some cheese .,ate
2,i ate *some* cheese .,the
3,i *ate* some cheese .,eat
4,it *'s* monday .,is
...,...,...
62595,she watched him *drawing* a picture .,draw
62596,what 's your favorite *horror* movie ?,scary
62597,what 's your favorite *scary* movie ?,horror
62598,*globalise* or die .,globalize


In [ ]:
patternDel = "\w\*\w\*\w"
filter = df_paraphrases['sentence'].str.contains(patternDel)

0        False
1        False
2        False
3        False
4        False
         ...  
62595    False
62596    False
62597    False
62598    False
62599    False
Name: sentence, Length: 62600, dtype: bool

In [ ]:
df_paraphrases = df_paraphrases[~filter]

In [ ]:
df_paraphrases

,sentence,paraphrase_word
0,i ate *the* cheese .,some
1,i *eat* some cheese .,ate
2,i ate *some* cheese .,the
3,i *ate* some cheese .,eat
4,it *'s* monday .,is
...,...,...
62595,she watched him *drawing* a picture .,draw
62596,what 's your favorite *horror* movie ?,scary
62597,what 's your favorite *scary* movie ?,horror
62598,*globalise* or die .,globalize


In [ ]:
from google.colab import drive
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [ ]:
df_paraphrases.to_csv('trainingdata_paraphrases.csv')
!cp trainingdata_paraphrases.csv "/content/drive/MyDrive/Colab Notebooks/"